In [1]:

%load_ext autoreload
%autoreload 2

import particle as pt
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mplhep
from scipy.stats import norm
from scipy.optimize import curve_fit
import matplotlib.mlab as mlab
import time
from matplotlib.colors import LogNorm
mplhep.style.use(mplhep.style.ROOT)
from pprint import pprint
from tqdm import tqdm
import polars as pl
import sys
import os
import particle
# using getlogin() returning username
user_name = os.getlogin()

sys.path.append(f'/home/{user_name}/Documents/cernbox/Atmos_Pandora/apc_atmo_repo/Anatree/')
from anatree_class import Anatree
from  ana_tools import *


plt.rcParams.update({'font.size': 16,
                     'grid.linestyle': '--',
                     'axes.grid': True,
                     'figure.autolayout': True,
                     'figure.figsize': [14,6]
                     })

anatree:Anatree

def selection_events(extras = ['']):
    r = ['subrun', 'event']
    if extras != ['']:
        r = r + extras
    return r
def get_event(subrun=0, event=1):
    return (pl.col('subrun')==subrun) & (pl.col('event') == event)

In [2]:
anatree = Anatree("/home/henrique/Documents/cernbox/Atmos_Pandora/data/atm_50k_hd_AV_2.5_random/anatree_hd_AV_2dot5_random_sum_300k.root", entry_start=0, entry_stop=1000, load_data=False)
anatree.tree:uproot.TTree
anatree.read_parquet('../../../../../data/atm_50k_hd_AV_2.5_random/parquet',batches=-1, concat=False)#, types=['nu','reco_tracks','reco_showers'])#, n_rows=100)

Reading nu files... 6/6
Reading geant files... 6/6
Reading reco_tracks files... 6/6
Reading reco_showers files... 6/6



In [3]:
nu = anatree.nu
geant = anatree.geant
shw = anatree.reco_showers
tracks = anatree.reco_tracks
showers = anatree.reco_showers
merged = 0

shw = 0

for i,n in enumerate(nu):
    df = anatree.get_full_reco_tracks(tracks[i], geant[i], nu[i])
    dfs = showers[i]
    # df = df.head(2000)
    df = df.collect()
    dfs = dfs.collect()
    if i == 0:
        merged = df
        shw = dfs
    else:
        merged = pl.concat([merged,df])
        shw = pl.concat([shw,dfs])

In [4]:
merged:pl.DataFrame
shw:pl.DataFrame
def get_tracks(df, subrun, event, coord='x'):
    return df.filter(get_event(subrun,event)).select(
        pl.col(f'trkstart{coord}_pandoraTrack'),
        pl.col(f'trkend{coord}_pandoraTrack'),
        pl.col('trkId_pandoraTrack'),
        pl.col('pdg_geant').apply(lambda x: particle.Particle.from_pdgid(x).latex_name)
        ).to_numpy()
def get_showers(df, subrun, event, coord='x'):
    return df.filter(get_event(subrun,event)).select(
        pl.col(f'shwr_start{coord}_pandoraShower'),
        pl.col(f'shwr_length_pandoraShower')*pl.col(f'shwr_startdcos{coord}_pandoraShower')+pl.col(f'shwr_start{coord}_pandoraShower'),
        ).filter(
            pl.col(f'shwr_start{coord}_pandoraShower') != -999
        ).to_numpy()
# showers[0].filter(get_event(subrun,event)).collect()

In [10]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

def extract_id_pdg(v):
    pdg = v[-1]
    v = np.delete(v,-1)
    id = v[-1]
    v = np.delete(v,-1)
    return v, id, pdg


data = []
subrun=3811
event=14

subrun=0
event=9

subrun=146
event=10
tx = get_tracks(merged,subrun,event,'x')
ty = get_tracks(merged,subrun,event,'y')
tz = get_tracks(merged,subrun,event,'z')

# sx = get_showers(shwf,subrun,event,'x')
# sy = get_showers(shwf,subrun,event,'y')
# sz = get_showers(shwf,subrun,event,'z')

for x,y,z in zip(tx, ty, tz):
    x, id, pdg = extract_id_pdg(x)
    y, id, pdg = extract_id_pdg(y)
    z, id, pdg = extract_id_pdg(z)
    data.append(
        go.Scatter3d(x=x, y=y, z=z,
                     mode='lines',
                     line={
                         'width': 10,
                         'dash': "dash"
                     },
                     name=fr"$id: {id}, pdg: {pdg}$",
                     showlegend=True,
                     textfont={
                         'size':20
                     }
                     
                     )
    )

# for i, (x,y,z) in enumerate(zip(sx, sy, sz)):
#     data.append(
#         go.Scatter3d(x=x, y=y, z=z,
#                      mode='lines',
#                      line={
#                          'width': 15,
#                      },
#                      name=fr"$id: {i}, (shower)$",
#                      opacity= 0.5,
#                      showlegend=True,
#                      textfont={
#                          'size':20
#                      }
                     
#                      )
#     )
layout = go.Layout(
    width=1200,
    margin={'l': 1, 'r': 1, 'b': 1, 't': 1},
    legend=dict(title_font_family="Times New Roman",
                              font=dict(size= 18))
)
plot_figure = go.Figure(data=data, layout=layout)
plot_figure.show()

In [8]:
showers[3].filter(get_event(3811,8)).collect()
merged.filter(get_event(3811,8)).select('trklen_pandoraTrack')
slen = np.sqrt((sx[:,1] - sx[:,0])**2  + (sy[:,1] - sy[:,0])**2  + (sz[:,1] - sz[:,0])**2)

slen

NameError: name 'sx' is not defined

In [9]:
shw.filter(get_event(0,4)).select(
    pl.col('^shwr_.*$')
)

shwr_bestplane_pandoraShower,shwr_length_pandoraShower,shwr_startdcosx_pandoraShower,shwr_startdcosy_pandoraShower,shwr_startdcosz_pandoraShower,shwr_startx_pandoraShower,shwr_starty_pandoraShower,shwr_startz_pandoraShower,shwr_totEng_pandoraShower_x,shwr_totEng_pandoraShower_y,shwr_totEng_pandoraShower_z,shwr_dedx_pandoraShower_x,shwr_dedx_pandoraShower_y,shwr_dedx_pandoraShower_z,shwr_mipEng_pandoraShower_x,shwr_mipEng_pandoraShower_y,shwr_mipEng_pandoraShower_z,shwr_hasPFParticle_pandoraShower,shwr_PFParticleID_pandoraShower,shwr_pidmvamu_pandoraShower,shwr_pidmvae_pandoraShower,shwr_pidmvapich_pandoraShower,shwr_pidmvaphoton_pandoraShower,shwr_pidmvapr_pandoraShower
i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i16,f32,f32,f32,f32,f32
1,96.466362,0.232655,0.909402,0.344761,37.694302,-406.247253,137.840637,464.195648,443.998779,451.559448,5.887189,9.460866,10.982842,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
1,16.136236,-0.509681,0.848531,-0.1422,122.727028,-275.15802,204.328583,107.303009,126.432251,106.487022,2.626674,3.108196,2.307724,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
0,5.87562,0.249815,0.784212,-0.567983,114.148003,-259.015442,201.038361,64.419044,-999.0,61.740231,9.92446,-999.0,9.538828,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,8.52058,24.656363,-999.0,-999.0,-999.0,-999.0,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0


In [10]:
shwf = shw.filter(
    pl.col('shwr_startx_pandoraShower') != -999
).filter(
    pl.col('shwr_starty_pandoraShower') != -999
).filter(
    pl.col('shwr_startz_pandoraShower') != -999
).filter(
    (pl.col('shwr_totEng_pandoraShower_x') != -999) | (pl.col('shwr_totEng_pandoraShower_y') != -999) | (pl.col('shwr_totEng_pandoraShower_z') != -999)
# ).filter(
#     True
    # (pl.col('shwr_dedx_pandoraShower_x') != -999) & (pl.col('shwr_dedx_pandoraShower_y') != -999) & (pl.col('shwr_dedx_pandoraShower_z') != -999)
).filter(
    pl.col('shwr_length_pandoraShower') != -999
)

len(shwf)-len(merged)

129

In [12]:
t = merged.groupby(selection_events()).agg(
    pl.col('trkId_pandoraTrack').count()
).sort(selection_events())
s = shw.groupby(selection_events()).agg(
    pl.col('showerID_pandoraShower').count()
).sort(selection_events())

n = t.join(s,left_on=selection_events(), right_on=selection_events())

nfilter = n.with_columns(
    diff = (pl.col('trkId_pandoraTrack').cast(pl.Int32) - pl.col('showerID_pandoraShower').cast(pl.Int32))
).filter(
    pl.col('diff') != 0
)
shw.join(nfilter,left_on=selection_events(), right_on=selection_events())

# t.groupby(selection_events(['trkId_pandoraTrack'])).agg(

# ).sort(selection_events())

subrun,event,shwr_bestplane_pandoraShower,shwr_length_pandoraShower,shwr_startdcosx_pandoraShower,shwr_startdcosy_pandoraShower,shwr_startdcosz_pandoraShower,shwr_startx_pandoraShower,shwr_starty_pandoraShower,shwr_startz_pandoraShower,shwr_totEng_pandoraShower_x,shwr_totEng_pandoraShower_y,shwr_totEng_pandoraShower_z,shwr_dedx_pandoraShower_x,shwr_dedx_pandoraShower_y,shwr_dedx_pandoraShower_z,shwr_mipEng_pandoraShower_x,shwr_mipEng_pandoraShower_y,shwr_mipEng_pandoraShower_z,shwr_hasPFParticle_pandoraShower,shwr_PFParticleID_pandoraShower,shwr_pidmvamu_pandoraShower,shwr_pidmvae_pandoraShower,shwr_pidmvapich_pandoraShower,shwr_pidmvaphoton_pandoraShower,shwr_pidmvapr_pandoraShower,showerID_pandoraShower,trkId_pandoraTrack,showerID_pandoraShower_right,diff
i32,i32,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i16,f32,f32,f32,f32,f32,i16,u32,u32,i32
0,4,1,96.466362,0.232655,0.909402,0.344761,37.694302,-406.247253,137.840637,464.195648,443.998779,451.559448,5.887189,9.460866,10.982842,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,3,4,-1
0,4,1,16.136236,-0.509681,0.848531,-0.1422,122.727028,-275.15802,204.328583,107.303009,126.432251,106.487022,2.626674,3.108196,2.307724,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,3,4,-1
0,4,0,5.87562,0.249815,0.784212,-0.567983,114.148003,-259.015442,201.038361,64.419044,-999.0,61.740231,9.92446,-999.0,9.538828,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,3,4,-1
0,4,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,8.52058,24.656363,-999.0,-999.0,-999.0,-999.0,-99999.0,-99999.0,-99999.0,1,3,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,3,4,-1
0,5,0,24.156116,-0.226436,-0.57362,0.787202,92.810913,-494.948029,347.923615,94.819046,91.221069,91.665482,2.29058,2.629945,2.168712,-99999.0,-99999.0,-99999.0,1,1,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,4,5,-1
0,5,0,9.15351,-0.842969,-0.392632,0.367755,48.093544,-491.475372,350.72171,46.294029,45.734295,51.824993,4.087903,2.750652,1.782876,-99999.0,-99999.0,-99999.0,1,1,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,4,5,-1
0,5,1,9.786713,0.196534,0.752078,0.629089,93.6007,-490.392273,349.062988,64.850342,61.339409,38.52132,2.676831,7.194021,2.474085,-99999.0,-99999.0,-99999.0,1,1,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,4,5,-1
0,5,0,5.850799,-0.972875,0.079743,-0.217152,91.186913,-494.322784,347.532867,51.982449,61.798157,54.32618,3.52661,4.127992,15.524368,-99999.0,-99999.0,-99999.0,1,1,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,4,5,-1
0,5,2,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.743419,23.67831,-999.0,-999.0,-999.0,-99999.0,-99999.0,-99999.0,1,1,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-999,4,5,-1


In [23]:
merged.filter(get_event(7470,10)).sort('trklen_pandoraTrack').select(['pdg_geant', 'Eng_geant', 'trklen_pandoraTrack'])

pdg_geant,Eng_geant,trklen_pandoraTrack
i32,f32,f32
2212,1.088234,23.251923
2212,1.275908,49.70554
13,0.375124,109.910606


In [20]:
merged['Eng_geant'].describe()

statistic,value
str,f64
"""count""",854028.0
"""null_count""",0.0
"""mean""",1.748557
"""std""",5.395938
"""min""",0.000002
"""max""",98.411682
"""median""",0.979221
"""25%""",0.271389
"""75%""",1.154097
